In [2]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from urllib.request import urlretrieve
from os import listdir
from os.path import isfile, join, exists
from zipfile import ZipFile
##for random split of dataset into a training set and a test set ###
from sklearn.model_selection import train_test_split
## image processing routines for feature extraction/transformation##
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Download Dataset
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Downloading SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Downloaded SceneClass13.rar')
    print ('Extracting SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Extracted SceneClass13.rar')
else:
    print ('Dataset already downloaded and extracted!')


#Get all the filenames (including the full path) in a folder as a list.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# local descriptor daisy and  histogram of oriented gradients (HOG)
#Function to extract daisy features as well as hog features from an image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #calculate daisy feature descriptors
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" ##path where image data is kept
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Load file names corresponding to each scene category into lists
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] ##list to keep path of all files in the database
dataset_labels=[]
##category_names.index('store')  list the numeric representation of the category
##category_names[0] list the text representation of the category id
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('total dataset size:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('total files in files :',len(dataset_filenames))
print ('total files in data set:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('total files in training split:',len(train_filenames))
print ('total files in testing split:',len(test_filenames))
#Extract features from training datasplit for downstream processing, takes aprox 12 mins for a standard laptop
training_data_feature_map={} ##map to store daisy feature as well as hog feature for all training datapoints
daisy_descriptor_list=[] ##list to store all daisy descriptors to form our visual vocabulary by clustering
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    ###extract DAISY features and HOG features from the image and save in a map###
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Now to form "visual-words", we cluster DAISY features to form a vocabulary, we form a histogram feature (daisy histogram) corresponding to each feature of 'number_of_clusters' dimensions

#input: list of DAISY features and number of clusters
#output: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
### hide warnings ##
import warnings
warnings.filterwarnings('ignore')
#Number of clusters is set as 600#,takes aprox 10 mins to run on standard laptop
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Function to extract hybrid feature from images by pooling daisy histogram and hog descriptor after individual
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #incase if we have encountered the file during training, the daisy and hog features would already have been computed
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    ##find to which clusters each daisy feature belongs
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Training data feature extraction
XTRAIN=[]
YTRAIN=[]
for i in tqdm(range(len(train_filenames))):
    XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
    YTRAIN.append(train_labels[i])
hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #cross-validated hyper-parameters
hybrid_classifier.fit(XTRAIN,YTRAIN)

### show a sample classification ###
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[3],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

### testing data feature extraction ###, do only for 200 instances if a quick test is required
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Hybrid Classifier Mertics')
print ('No. of test instances:',len(XTEST),len(YTEST))
### Accuracy Report ###
hybridpred=hybrid_classifier.predict(XTEST)
print ('Overall accuracy:',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Confusion matrix:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
             columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'])

Dataset already downloaded and extracted!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
total dataset size: 3859
total files in files : 3859
total files in data set: 3859
total files in training split: 1300
total files in testing split: 2559


  0%|                                                 | 0/1300 [00:00<?, ?it/s]C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
100%|██████████████████████████████████████| 1300/1300 [06:16<00:00,  2.82it/s]


Total daisy descriptors: 1404000


MemoryError: 

In [2]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from urllib.request import urlretrieve
from os import listdir
from os.path import isfile, join, exists
from zipfile import ZipFile
##for random split of dataset into a training set and a test set ###
from sklearn.model_selection import train_test_split
## image processing routines for feature extraction/transformation##
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Download Dataset
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Downloading SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Downloaded SceneClass13.rar')
    print ('Extracting SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Extracted SceneClass13.rar')
else:
    print ('Dataset already downloaded and extracted!')


#Get all the filenames (including the full path) in a folder as a list.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# local descriptor daisy and  histogram of oriented gradients (HOG)
#Function to extract daisy features as well as hog features from an image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #calculate daisy feature descriptors
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" ##path where image data is kept
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Load file names corresponding to each scene category into lists
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] ##list to keep path of all files in the database
dataset_labels=[]
##category_names.index('store')  list the numeric representation of the category
##category_names[0] list the text representation of the category id
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('total dataset size:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('total files in files :',len(dataset_filenames))
print ('total files in data set:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=600, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('total files in training split:',len(train_filenames))
print ('total files in testing split:',len(test_filenames))
#Extract features from training datasplit for downstream processing, takes aprox 12 mins for a standard laptop
training_data_feature_map={} ##map to store daisy feature as well as hog feature for all training datapoints
daisy_descriptor_list=[] ##list to store all daisy descriptors to form our visual vocabulary by clustering
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    ###extract DAISY features and HOG features from the image and save in a map###
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Now to form "visual-words", we cluster DAISY features to form a vocabulary, we form a histogram feature (daisy histogram) corresponding to each feature of 'number_of_clusters' dimensions

#input: list of DAISY features and number of clusters
#output: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
### hide warnings ##
import warnings
warnings.filterwarnings('ignore')
#Number of clusters is set as 600#,takes aprox 10 mins to run on standard laptop
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Function to extract hybrid feature from images by pooling daisy histogram and hog descriptor after individual
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #incase if we have encountered the file during training, the daisy and hog features would already have been computed
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    ##find to which clusters each daisy feature belongs
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Training data feature extraction
XTRAIN=[]
YTRAIN=[]
for i in tqdm(range(len(train_filenames))):
    XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
    YTRAIN.append(train_labels[i])
hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #cross-validated hyper-parameters
hybrid_classifier.fit(XTRAIN,YTRAIN)

### show a sample classification ###
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[3],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

### testing data feature extraction ###, do only for 200 instances if a quick test is required
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Hybrid Classifier Mertics')
print ('No. of test instances:',len(XTEST),len(YTEST))
### Accuracy Report ###
hybridpred=hybrid_classifier.predict(XTEST)
print ('Overall accuracy:',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Confusion matrix:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
             columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'])

Dataset already downloaded and extracted!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
total dataset size: 3859
total files in files : 3859
total files in data set: 3859
total files in training split: 600
total files in testing split: 3259


  0%|                                                  | 0/600 [00:00<?, ?it/s]C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
100%|████████████████████████████████████████| 600/600 [02:58<00:00,  3.37it/s]


Total daisy descriptors: 648000


100%|████████████████████████████████████████| 600/600 [00:31<00:00, 18.78it/s]


true label: 9.0
prediction: [ 3.]


100%|██████████████████████████████████████| 3259/3259 [17:40<00:00,  3.23it/s]


Hybrid Classifier Mertics
No. of test instances: 3259 3259
Overall accuracy: 0.754219085609
                 precision    recall  f1-score   support

        bedroom       0.66      0.38      0.49       182
      CALsuburb       0.98      0.95      0.97       203
        kitchen       0.63      0.46      0.53       177
     livingroom       0.51      0.71      0.60       244
       MITcoast       0.74      0.82      0.78       304
      MITforest       0.87      0.92      0.89       277
     MIThighway       0.87      0.73      0.79       220
  MITinsidecity       0.78      0.68      0.73       260
    MITmountain       0.77      0.78      0.78       316
 MITopencountry       0.68      0.75      0.71       346
      MITstreet       0.86      0.80      0.83       247
MITtallbuilding       0.81      0.86      0.83       301
      PARoffice       0.71      0.76      0.73       182

    avg / total       0.76      0.75      0.75      3259

Confusion matrix:



MITinsidecity  bedroom  PARoffice  MITmountain  \
MITinsidecity               70        0         14           66   
bedroom                      0      193          0            1   
PARoffice                    9        0         82           55   
MITmountain                 23        0         15          174   
MITtallbuilding              0        0          0            0   
MIThighway                   0        2          0            0   
MITcoast                     2        0          0            0   
livingroom                   0        0          4            4   
MITopencountry               0        0          0            1   
MITstreet                    0        0          0            0   
MITforest                    0        1          0            3   
kitchen                      0        0          1           12   
CALsuburb                    2        0         14           23   

                 MITtallbuilding  MIThighway  MITcoast  livingroom  \
MITinsidecity                  3           0         0           0   
bedroom                        0           0         0           1   
PARoffice                      1           0         0          13   
MITmountain                    2           0         0           8   
MITtallbuilding              248           3         3           0   
MIThighway                     0         255         0           0   
MITcoast                      22           0       160           4   
livingroom                     5           2         4         177   
MITopencountry                17           9         3           0   
MITstreet                     32          17         7           0   
MITforest                      1           0         5          18   
kitchen                        3           8         1           6   
CALsuburb                      1           0         0           1   

                 MITopencountry  MITstreet  MITforest  kitchen  CALsuburb  
MITinsidecity                 2          1          1        4         21  
bedroom                       2          0          1        5          0  
PARoffice                     0          0          0        0         17  
MITmountain                   0          2          1        4         15  
MITtallbuilding               3         46          0        0          1  
MIThighway                   12          7          0        1          0  
MITcoast                      8         21          2        0          1  
livingroom                    2          9         22       30          1  
MITopencountry              247         34          2        3          0  
MITstreet                    31        258          1        0          0  
MITforest                     8          1        197       13          0  
kitchen                       6          2          2      258          2  
CALsuburb                     0          0          0        2        139

In [ ]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from urllib.request import urlretrieve
from os import listdir
from os.path import isfile, join, exists
from zipfile import ZipFile
##for random split of dataset into a training set and a test set ###
from sklearn.model_selection import train_test_split
## image processing routines for feature extraction/transformation##
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Download Dataset
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Downloading SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Downloaded SceneClass13.rar')
    print ('Extracting SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Extracted SceneClass13.rar')
else:
    print ('Dataset already downloaded and extracted!')


#Get all the filenames (including the full path) in a folder as a list.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# local descriptor daisy and  histogram of oriented gradients (HOG)
#Function to extract daisy features as well as hog features from an image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #calculate daisy feature descriptors
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" ##path where image data is kept
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Load file names corresponding to each scene category into lists
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] ##list to keep path of all files in the database
dataset_labels=[]
##category_names.index('store')  list the numeric representation of the category
##category_names[0] list the text representation of the category id
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('total dataset size:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('total files in files :',len(dataset_filenames))
print ('total files in data set:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('total files in training split:',len(train_filenames))
print ('total files in testing split:',len(test_filenames))
#Extract features from training datasplit for downstream processing, takes aprox 12 mins for a standard laptop
training_data_feature_map={} ##map to store daisy feature as well as hog feature for all training datapoints
daisy_descriptor_list=[] ##list to store all daisy descriptors to form our visual vocabulary by clustering
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    ###extract DAISY features and HOG features from the image and save in a map###
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Now to form "visual-words", we cluster DAISY features to form a vocabulary, we form a histogram feature (daisy histogram) corresponding to each feature of 'number_of_clusters' dimensions

#input: list of DAISY features and number of clusters
#output: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
### hide warnings ##
import warnings
warnings.filterwarnings('ignore')
#Number of clusters is set as 600#,takes aprox 10 mins to run on standard laptop
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Function to extract hybrid feature from images by pooling daisy histogram and hog descriptor after individual
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #incase if we have encountered the file during training, the daisy and hog features would already have been computed
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    ##find to which clusters each daisy feature belongs
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Training data feature extraction
XTRAIN=[]
YTRAIN=[]
for i in tqdm(range(len(train_filenames))):
    XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
    YTRAIN.append(train_labels[i])
hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #cross-validated hyper-parameters
hybrid_classifier.fit(XTRAIN,YTRAIN)

### show a sample classification ###
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[3],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

### testing data feature extraction ###, do only for 200 instances if a quick test is required
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Hybrid Classifier Mertics')
print ('No. of test instances:',len(XTEST),len(YTEST))
### Accuracy Report ###
hybridpred=hybrid_classifier.predict(XTEST)
print ('Overall accuracy:',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Confusion matrix:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
             columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'])

Dataset already downloaded and extracted!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
total dataset size: 3859
total files in files : 3859
total files in data set: 3859
total files in training split: 1300
total files in testing split: 2559


100%|██████████████████████████████████████| 1300/1300 [05:48<00:00,  3.60it/s]


Total daisy descriptors: 1404000


100%|██████████████████████████████████████| 1300/1300 [03:15<00:00,  6.66it/s]


true label: 4.0
prediction: [ 10.]


100%|██████████████████████████████████████| 2559/2559 [14:18<00:00,  3.09it/s]


Hybrid Classifier Mertics
No. of test instances: 2559 2559
Overall accuracy: 0.803048065651
                 precision    recall  f1-score   support

        bedroom       0.63      0.44      0.52       143
      CALsuburb       0.99      0.99      0.99       160
        kitchen       0.65      0.51      0.57       139
     livingroom       0.57      0.70      0.63       192
       MITcoast       0.80      0.84      0.82       239
      MITforest       0.92      0.94      0.93       217
     MIThighway       0.86      0.84      0.85       172
  MITinsidecity       0.78      0.82      0.80       204
    MITmountain       0.81      0.90      0.85       248
 MITopencountry       0.77      0.76      0.77       272
      MITstreet       0.92      0.81      0.86       194
MITtallbuilding       0.87      0.87      0.87       236
      PARoffice       0.79      0.83      0.81       143

    avg / total       0.80      0.80      0.80      2559

Confusion matrix:



MITinsidecity  bedroom  PARoffice  MITmountain  \
MITinsidecity               63        1         11           49   
bedroom                      0      158          0            0   
PARoffice                    9        0         71           35   
MITmountain                 20        0         12          135   
MITtallbuilding              0        0          0            0   
MIThighway                   0        0          0            0   
MITcoast                     0        0          0            0   
livingroom                   0        0          5            2   
MITopencountry               1        0          0            0   
MITstreet                    0        0          0            0   
MITforest                    0        1          0            3   
kitchen                      2        0          1            4   
CALsuburb                    5        0          9            9   

                 MITtallbuilding  MIThighway  MITcoast  livingroom  \
MITinsidecity                  3           0         1           2   
bedroom                        0           0         0           0   
PARoffice                      0           0         0          11   
MITmountain                    0           0         1           5   
MITtallbuilding              200           0         3           0   
MIThighway                     0         205         0           0   
MITcoast                       7           1       145           4   
livingroom                     1           1         6         167   
MITopencountry                 7           3         2           0   
MITstreet                     28          11         4           0   
MITforest                      0           0         6          13   
kitchen                        3           3         0          11   
CALsuburb                      0           0         0           0   

                 MITopencountry  MITstreet  MITforest  kitchen  CALsuburb  
MITinsidecity                 3          0          1        2          7  
bedroom                       0          0          1        1          0  
PARoffice                     0          0          1        3          9  
MITmountain                   1          1          0        4         13  
MITtallbuilding               3         33          0        0          0  
MIThighway                    9          3          0        0          0  
MITcoast                      6          8          1        0          0  
livingroom                    0          4          7       11          0  
MITopencountry              222         13          0        0          0  
MITstreet                    18        208          3        0          0  
MITforest                     6          0        158        7          0  
kitchen                       5          0          0      205          2  
CALsuburb                     0          0          0        2        118

In [2]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
#modèle entrainé
filename = 'trainedsvm.sav'
for i in tqdm(range(len(train_filenames))):
    XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
    YTRAIN.append(train_labels[i])
if os.path.exists(filename):
   # load the model from disk
    hybrid_classifier = joblib.load(filename) 
else:
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    joblib.dump(hybrid_classifier, filename)


# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[3],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride)
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
hybridpred=hybrid_classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
             columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

SyntaxError: EOL while scanning string literal (<ipython-input-2-ac4075f9bccc>, line 179)

In [ ]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
#modèle entrainé
filename = 'trainedsvm.sav'
for i in tqdm(range(len(train_filenames))):
    XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
    YTRAIN.append(train_labels[i])
if os.path.exists(filename):
   # load the model from disk
    hybrid_classifier = joblib.load(filename) 
else:
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    joblib.dump(hybrid_classifier, filename)


# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[3],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
hybridpred=hybrid_classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
             columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de donnéesd : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559


  0%|                                                 | 0/1300 [00:00<?, ?it/s]C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
100%|██████████████████████████████████████| 1300/1300 [07:19<00:00,  1.77it/s]


Total daisy descriptors: 1404000


In [6]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
#modèle entrainé
filename = 'trainedsvm.sav'
for i in tqdm(range(len(train_filenames))):
    XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
    YTRAIN.append(train_labels[i])
if os.path.exists(filename):
   # load the model from disk
    hybrid_classifier = joblib.load(filename) 
else:
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    joblib.dump(hybrid_classifier, filename)


# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[3],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
hybridpred=hybrid_classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
             columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de donnéesd : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559


100%|██████████████████████████████████████| 1300/1300 [06:22<00:00,  3.52it/s]


Total daisy descriptors: 1404000


100%|██████████████████████████████████████| 1300/1300 [08:24<00:00,  2.58it/s]


true label: 8.0
prediction: [ 11.]


100%|██████████████████████████████████████| 2559/2559 [15:23<00:00,  3.31it/s]


Métriques du classifieur hybride
Nombre instances test: 2559 2559
Précision globale: 
 0.431027745213
                 precision    recall  f1-score   support

        bedroom       0.65      0.37      0.47       143
      CALsuburb       1.00      0.09      0.17       160
        kitchen       0.60      0.40      0.48       139
     livingroom       0.32      0.72      0.44       192
       MITcoast       0.53      0.91      0.67       239
      MITforest       0.00      0.00      0.00       217
     MIThighway       0.69      0.67      0.68       172
  MITinsidecity       0.75      0.48      0.58       204
    MITmountain       0.43      0.60      0.50       248
 MITopencountry       0.68      0.12      0.21       272
      MITstreet       1.00      0.04      0.07       194
MITtallbuilding       0.44      0.61      0.51       236
      PARoffice       0.16      0.55      0.24       143

    avg / total       0.55      0.43      0.39      2559

Matrice de confusion:



MITinsidecity  bedroom  PARoffice  MITmountain  \
MITinsidecity               53        0          3           58   
bedroom                      0       15          8           18   
PARoffice                    3        0         55           42   
MITmountain                 10        0          5          138   
MITtallbuilding              0        0          0            3   
MIThighway                   0        0          0           15   
MITcoast                     3        0          1            9   
livingroom                   0        0          6           17   
MITopencountry               0        0          3           29   
MITstreet                    1        0          0            7   
MITforest                    4        0          2           49   
kitchen                      1        0          2           23   
CALsuburb                    6        0          7           22   

                 MITtallbuilding  MIThighway  MITcoast  livingroom  \
MITinsidecity                  1           0         0           0   
bedroom                        0           0         3           3   
PARoffice                      1           0         0           2   
MITmountain                    0           0         2           2   
MITtallbuilding              217           0        13           0   
MIThighway                    14           0         0           3   
MITcoast                      21           0       116           3   
livingroom                     3           0         1          97   
MITopencountry                15           0         9           0   
MITstreet                    130           0        22           2   
MITforest                      3           0         1           2   
kitchen                        5           0         0          12   
CALsuburb                      0           0         0           4   

                 MITopencountry  MITstreet  MITforest  kitchen  CALsuburb  
MITinsidecity                 0          0          0        6         22  
bedroom                       0          1          0       15         97  
PARoffice                     0          0          0       14         22  
MITmountain                   0          0          0        5         30  
MITtallbuilding               3          3          0        0          0  
MIThighway                  126          0          0       41         18  
MITcoast                      4          7          0        5          3  
livingroom                    0          0          0       45         35  
MITopencountry              148          4          0       10         30  
MITstreet                    61         34          0        6          9  
MITforest                     0          0          7       13        113  
kitchen                       0          1          0      144         48  
CALsuburb                     0          0          0       25         79

In [ ]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
#modèle entrainé
filename = 'trainedsvm.sav'
for i in tqdm(range(len(train_filenames))):
    XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
    YTRAIN.append(train_labels[i])
if os.path.exists(filename):
   # load the model from disk
    hybrid_classifier = joblib.load(filename) 
else:
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    joblib.dump(hybrid_classifier, filename)


# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[3],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
hybridpred=hybrid_classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
             columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

In [ ]:
# %load scenereclassification.py

#%load scenereclassification.py
import os.path
from sklearn.externals import joblib
from urllib.request import urlretrieve
from os import listdir
from os.path import isfile, join, exists
from zipfile import ZipFile
##for random split of dataset into a training set and a test set ###
from sklearn.model_selection import train_test_split
## image processing routines for feature extraction/transformation##
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Download Dataset
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Downloading SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Downloaded SceneClass13.rar')
    print ('Extracting SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Extracted SceneClass13.rar')
else:
    print ('Dataset already downloaded and extracted!')


#Get all the filenames (including the full path) in a folder as a list.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles

#Function to extract daisy features as well as hog features from an image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #calculate daisy feature descriptors
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" ##path where image data is kept
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Load file names corresponding to each scene category into lists
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] ##list to keep path of all files in the database
dataset_labels=[]
##category_names.index('store')  list the numeric representation of the category
##category_names[0] list the text representation of the category id
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
print ('total dataset size:',len(dataset_filenames))
#Split into training files and testing files
train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,test_size=0.6,random_state=42)
print ('total files in training split:',len(train_filenames))
print ('total files in testing split:',len(test_filenames))
#Extract features from training datasplit for downstream processing, takes aprox 12 mins for a standard laptop
training_data_feature_map={} ##map to store daisy feature as well as hog feature for all training datapoints
daisy_descriptor_list=[] ##list to store all daisy descriptors to form our visual vocabulary by clustering
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    ###extract DAISY features and HOG features from the image and save in a map###
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Now to form "visual-words", we cluster DAISY features to form a vocabulary, we form a histogram feature (daisy histogram) corresponding to each feature of 'number_of_clusters' dimensions

#input: list of DAISY features and number of clusters
#output: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
### hide warnings ##
import warnings
warnings.filterwarnings('ignore')
#Number of clusters is set as 600#,takes aprox 10 mins to run on standard laptop
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Function to extract hybrid feature from images by pooling daisy histogram and hog descriptor after individual
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #incase if we have encountered the file during training, the daisy and hog features would already have been computed
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    ##find to which clusters each daisy feature belongs
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Training data feature extraction
XTRAIN=[]
YTRAIN=[]
for i in tqdm(range(len(train_filenames))):
    XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
    YTRAIN.append(train_labels[i])
if os.path.exists(filename):
   # load the model from disk
    hybrid_classifier = joblib.load(filename) 
else:
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    joblib.dump(hybrid_classifier, filename)

### show a sample classification ###
plot_file(test_filenames[3])
print ('true label:',test_labels[3])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[3],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

### testing data feature extraction ###, do only for 200 instances if a quick test is required
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Hybrid Classifier Metrics')
print ('No. of test instances:',len(XTEST),len(YTEST))
### Accuracy Report ###
hybridpred=hybrid_classifier.predict(XTEST)
print ('Overall accuracy:',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Confusion matrix:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
             columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'])

Dataset already downloaded and extracted!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
total dataset size: 3859
total files in training split: 1543
total files in testing split: 2316


100%|██████████████████████████████████████| 1543/1543 [07:30<00:00,  2.78it/s]


Total daisy descriptors: 1666440


100%|██████████████████████████████████████| 1543/1543 [09:51<00:00,  3.85it/s]


true label: 8.0
prediction: [ 11.]


 15%|█████▋                                 | 336/2316 [01:52<10:54,  3.03it/s]

In [2]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    hybrid_classifier = joblib.load(filename) 
    print ('...Chargement du fichier apprentissage...')
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[3],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
hybridpred=hybrid_classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
             columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de donnéesd : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559


  0%|                                                 | 0/1300 [00:00<?, ?it/s]C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
100%|██████████████████████████████████████| 1300/1300 [07:14<00:00,  3.14it/s]


Total daisy descriptors: 1404000


MemoryError: 

In [4]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    hybrid_classifier = joblib.load(filename) 
    print ('...Chargement du fichier apprentissage...')
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
#for i in tqdm(range(len(test_filenames))):
#    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
#    YTEST.append(test_labels[i])
#print ('Métriques du classifieur hybride')
#print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
#hybridpred=hybrid_classifier.predict(XTEST)
       
#print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
#print (classification_report(YTEST, hybridpred, target_names=category_names))
#print ('Matrice de confusion:\n')
#pd.DataFrame(confusion_matrix(YTEST,hybridpred),
#            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
#               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de donnéesd : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559


 39%|███████████████▎                       | 509/1300 [02:43<04:38,  2.84it/s]

KeyboardInterrupt: 

In [6]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    hybrid_classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
#for i in tqdm(range(len(test_filenames))):
#    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
#    YTEST.append(test_labels[i])
#print ('Métriques du classifieur hybride')
#print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
#hybridpred=hybrid_classifier.predict(XTEST)
       
#print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
#print (classification_report(YTEST, hybridpred, target_names=category_names))
#print ('Matrice de confusion:\n')
#pd.DataFrame(confusion_matrix(YTEST,hybridpred),
#            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
#               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'])

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de donnéesd : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559



 84%|████████████████████████████████      | 1098/1300 [05:44<01:19,  2.54it/s]

KeyboardInterrupt: 

In [8]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
#for fname in tqdm(train_filenames):
#    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
#    training_data_feature_map[fname]=[daisy_features,hog_feature]
#    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
#print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    hybrid_classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
#for i in tqdm(range(len(test_filenames))):
#    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
#    YTEST.append(test_labels[i])
#print ('Métriques du classifieur hybride')
#print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
#hybridpred=hybrid_classifier.predict(XTEST)
       
#print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
#print (classification_report(YTEST, hybridpred, target_names=category_names))
#print ('Matrice de confusion:\n')
#pd.DataFrame(confusion_matrix(YTEST,hybridpred),
#            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
#               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de donnéesd : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.

In [10]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
#for fname in tqdm(train_filenames):
#    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
#    training_data_feature_map[fname]=[daisy_features,hog_feature]
#    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
#print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
#daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
#daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    hybrid_classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
#for i in tqdm(range(len(test_filenames))):
#    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
#    YTEST.append(test_labels[i])
#print ('Métriques du classifieur hybride')
#print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
#hybridpred=hybrid_classifier.predict(XTEST)
       
#print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
#print (classification_report(YTEST, hybridpred, target_names=category_names))
#print ('Matrice de confusion:\n')
#pd.DataFrame(confusion_matrix(YTEST,hybridpred),
#            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
#               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de donnéesd : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559
...Chargement du fichier apprentissage...
true label: 8.0


NameError: name 'daisy_cluster_model' is not defined

In [ ]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de donnéesd :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
daisy_cluster_model.n_clusters
#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    hybrid_classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
#for i in tqdm(range(len(test_filenames))):
#    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
#    YTEST.append(test_labels[i])
#print ('Métriques du classifieur hybride')
#print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
#hybridpred=hybrid_classifier.predict(XTEST)
       
#print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
#print (classification_report(YTEST, hybridpred, target_names=category_names))
#print ('Matrice de confusion:\n')
#pd.DataFrame(confusion_matrix(YTEST,hybridpred),
#            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
#               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'])

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de donnéesd : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559



100%|██████████████████████████████████████| 1300/1300 [06:04<00:00,  3.55it/s]


Total daisy descriptors: 1404000


In [2]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
for fname in tqdm(train_filenames):
    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
    training_data_feature_map[fname]=[daisy_features,hog_feature]
    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
  
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    hybrid_classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
#for i in tqdm(range(len(test_filenames))):
#    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
#    YTEST.append(test_labels[i])
#print ('Métriques du classifieur hybride')
#print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
#hybridpred=hybrid_classifier.predict(XTEST)
       
#print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
#print (classification_report(YTEST, hybridpred, target_names=category_names))
#print ('Matrice de confusion:\n')
#pd.DataFrame(confusion_matrix(YTEST,hybridpred),
#            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
#               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de données : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559


  0%|                                                 | 0/1300 [00:00<?, ?it/s]C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\helen fost\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
 27%|██████████▎                            | 345/1300 [01:31<04:27,  3.58it/s]

KeyboardInterrupt: 

In [4]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0
#for fname in tqdm(train_filenames):
#    daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
#    training_data_feature_map[fname]=[daisy_features,hog_feature]
#    daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
  
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    hybrid_classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
#for i in tqdm(range(len(test_filenames))):
#    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
#    YTEST.append(test_labels[i])
#print ('Métriques du classifieur hybride')
#print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
#hybridpred=hybrid_classifier.predict(XTEST)
       
#print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
#print (classification_report(YTEST, hybridpred, target_names=category_names))
#print ('Matrice de confusion:\n')
#pd.DataFrame(confusion_matrix(YTEST,hybridpred),
#            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
#               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de données : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559
Total daisy descriptors: 0
...Enregistrement du modèle...


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.

In [6]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0

#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
    for fname in tqdm(train_filenames):
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
        training_data_feature_map[fname]=[daisy_features,hog_feature]
        daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
print ('Total daisy descriptors:',len(daisy_descriptor_list))
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    hybrid_classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
#for i in tqdm(range(len(test_filenames))):
#    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
#    YTEST.append(test_labels[i])
#print ('Métriques du classifieur hybride')
#print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
#hybridpred=hybrid_classifier.predict(XTEST)
       
#print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
#print (classification_report(YTEST, hybridpred, target_names=category_names))
#print ('Matrice de confusion:\n')
#pd.DataFrame(confusion_matrix(YTEST,hybridpred),
#            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
#               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

IndentationError: unexpected indent (<ipython-input-6-fa49519b201e>, line 133)

In [8]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0

#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
    for fname in tqdm(train_filenames):
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
        training_data_feature_map[fname]=[daisy_features,hog_feature]
        daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
    print ('Total daisy descriptors:',len(daisy_descriptor_list))
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    hybrid_classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
#for i in tqdm(range(len(test_filenames))):
#    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
#    YTEST.append(test_labels[i])
#print ('Métriques du classifieur hybride')
#print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
#hybridpred=hybrid_classifier.predict(XTEST)
       
#print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
#print (classification_report(YTEST, hybridpred, target_names=category_names))
#print ('Matrice de confusion:\n')
#pd.DataFrame(confusion_matrix(YTEST,hybridpred),
#            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
#               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de données : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559



100%|██████████████████████████████████████| 1300/1300 [06:10<00:00,  3.33it/s]


Total daisy descriptors: 1404000
...Enregistrement du modèle...
...Chargement du fichier apprentissage...
true label: 11.0
prediction: [ 11.]


In [10]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local DAISY et histogramme de gradients orientés (HOG)
#Fonction pour extraire les fonctionnalités de DAISY ainsi que les fonctions de HOG à partir d'une image
def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
    hog_desriptor=hog(img, orientations=8, pixels_per_cell=(hog_pixels_per_cell, hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block), visualise=False,feature_vector=True)
    return daisy_desriptors,hog_desriptor


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0

#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend environ 10 minutes pour fonctionner sur un ordinateur portable standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
    for fname in tqdm(train_filenames):
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
        training_data_feature_map[fname]=[daisy_features,hog_feature]
        daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
    print ('Total daisy descriptors:',len(daisy_descriptor_list))
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_hog_hybrid_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    hybrid_classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_hog_hybrid_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    hybrid_classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    hybrid_classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(hybrid_classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_hog_hybrid_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',hybrid_classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
hybridpred=hybrid_classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,hybridpred),
            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de données : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559
...Chargement du fichier de kmean...
...Chargement du fichier apprentissage...
true label: 7.0
prediction: [ 12.]


  5%|██                                     | 132/2559 [00:41<12:12,  3.32it/s]

KeyboardInterrupt: 